## Install OpenPose

In [19]:
! apt update
! apt install -y cmake sudo libopencv-dev
#! git clone https://github.com/CMU-Perceptual-Computing-Lab/openpose.git
! cd openpose/ubuntu && ./install_cmake.sh && ./install_cuda.sh && ./install_cudnn.sh
! cd openpose && git pull origin master && rm -r build || true && mkdir build && cd build && cmake .. && make -j`nproc`

# example demo usage
!cd openpose && ./build/examples/openpose/openpose.bin --video examples/media/video.avi --write_json output/ --display 0 --render_pose 0

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,012 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,779 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restric

In [18]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [23]:
import os, sys

# Use the right Python version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

# install new CMake because of CUDA10
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
!git clone $git_repo_url
!sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
# install system dependencies
!apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
# build openpose
!cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
# Install body_25b model
!wget posefs1.perception.cs.cmu.edu/OpenPose/models/pose/1_25BBkg/body_25b/pose_iter_XXXXXX.caffemodel -P /content/openpose/models/pose/body_25b
!wget https://raw.githubusercontent.com/CMU-Perceptual-Computing-Lab/openpose_train/master/experimental_models/1_25BBkg/body_25b/pose_deploy.prototxt -P /content/openpose/models/pose/body_25b
# Get compatible CUDA and CuDNN versions in order to avoid `CudaSuccess (2 vs. 0) out of memory` error
!apt -qq install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
# Copy libraries in system path
!cp build/src/openpose/libopenpose.so.1.7.0 /usr/local/lib
!cp build/caffe/lib/libcaffe.so.1.0.0 /usr/local/lib
!sudo ldconfig

Cloning into 'openpose'...
remote: Enumerating objects: 16100, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 16100 (delta 0), reused 7 (delta 0), pack-reused 16090
Receiving objects: 100% (16100/16100), 84.45 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (11279/11279), done.
-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, addi

In [1]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`


Selecting previously unselected package libgflags2.2.
(Reading database ... 122541 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.2-1build1_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-1build1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.2-1build1_amd64.deb ...
Unpacking libgflags-dev (2.2.2-1build1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.4.0-1build1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.4.0-1build1) ...
Selecting previously unselected package libleveldb1d:amd64.
Preparing to unpack .../04-libleveldb1d_1.22-3ubuntu2_amd64.deb ...
Unpacking libleveldb1d:amd64 (1.22-3ubuntu2) ...
Selecting previously unselected package libleveldb-dev

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!mkdir videos
!cp -r "gdrive/MyDrive/Colab Notebooks/videos/." videos

mkdir: cannot create directory ‘videos’: File exists


In [8]:
!mkdir output

In [11]:
!apt list --installed libnvidia*
!apt list --installed "libcudnn*"
!apt list --installed cuda-cudart*

Listing... Done
Listing... Done
libcudnn8-dev/unknown,now 8.7.0.84-1+cuda11.8 amd64 [installed,upgradable to: 8.9.2.26-1+cuda12.1]
libcudnn8/unknown,now 8.7.0.84-1+cuda11.8 amd64 [installed,upgradable to: 8.9.2.26-1+cuda12.1]
Listing... Done
cuda-cudart-11-8/unknown,now 11.8.89-1 amd64 [installed]
cuda-cudart-dev-11-8/unknown,now 11.8.89-1 amd64 [installed]


In [12]:
cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
!tar xfz {cmake_version} --strip-components=1 -C /usr/local

In [16]:
!cmake -DUSE_CUDNN=OFF
#cmake -DPYBIND11_INSTALL=ON -DUSE_PYTHON_INCLUDE_DIR=ON -DGPU_MODE=CUDA -DUSE_CUDNN=ON -DBUILD_PYTHON=ON -DPYTHON_EXECUTABLE=/usr/bin/python3 ..

CMake Warning:
  No source or binary directory provided.  Both will be assumed to be the
  same as the current working directory, but note that this warning will
  become a fatal error in future CMake releases.


-- The C compiler identification is GNU 9.4.0
-- The CXX compiler identification is GNU 9.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- GCC detected, adding compile flags
-- GCC detected, adding compile flags
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Failed
-- Looking for pthread_create in pthreads
-- Looking fo

In [25]:
import os
os.chdir('/content/openpose')
!build/examples/openpose/openpose.bin --video ../videos/clip_marathon_1.mp4 --write_json ../output/ --display 0  --write_video ../output/openpose_clip_marathon_1.avi  #--net_resolution 320x176 --face_net_resolution 320x320
# convert the result into MP4
#!ffmpeg -y -loglevel info -i ../output/openpose_clip_marathon_1.avi ../output/openpose_clip_marathon_1.mp4

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 25.681103 seconds.


In [43]:
import os
import subprocess

def process_videos(directory, directory_output):
  for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print("Processing file: " + f)
        filepath, file_extension = os.path.splitext(f)
        filename = filepath.split("/")[-1]
        full_directory_output = directory_output + '/' + filename

        # Check if directory is available
        isExist = os.path.exists(full_directory_output)
        if not isExist:
          os.makedirs(full_directory_output)
          print("Created directory: " + full_directory_output)

        #cmd = 'python scripts/demo_inference.py --cfg {} --checkpoint {} --video {} --outdir "{}" --format open --save_video'
        cmd = 'build/examples/openpose/openpose.bin --video {} --write_json {} --display 0  --write_video {}/{}.avi'
        cmd = cmd.format(f, full_directory_output, full_directory_output, filename)
        print(cmd)

        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
        out, err = p.communicate()



In [44]:
process_videos("../videos", "../output")

Processing file: ../videos/clip_10k_2.mp4
build/examples/openpose/openpose.bin --video ../videos/clip_10k_2.mp4 --write_json ../output/clip_10k_2 --display 0  --write_video ../output/clip_10k_2/clip_10k_2.avi
Processing file: ../videos/clip_marathon_1.mp4
build/examples/openpose/openpose.bin --video ../videos/clip_marathon_1.mp4 --write_json ../output/clip_marathon_1 --display 0  --write_video ../output/clip_marathon_1/clip_marathon_1.avi
Processing file: ../videos/clip_tri_3.mp4
build/examples/openpose/openpose.bin --video ../videos/clip_tri_3.mp4 --write_json ../output/clip_tri_3 --display 0  --write_video ../output/clip_tri_3/clip_tri_3.avi
Processing file: ../videos/short.mp4
build/examples/openpose/openpose.bin --video ../videos/short.mp4 --write_json ../output/short --display 0  --write_video ../output/short/short.avi
Processing file: ../videos/manifest.json
build/examples/openpose/openpose.bin --video ../videos/manifest.json --write_json ../output/manifest --display 0  --write_v

In [45]:
!cp -r ../output/. "../gdrive/MyDrive/Colab Notebooks/OpenPose"

Finally, visualize the result:

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)